baxterのmap求める

In [4]:
import sympy as sy
from sympy import sin, cos, pi, sqrt
import math
q = sy.Matrix(sy.MatrixSymbol('q', 7, 1))
#L, h, H, L0, L1, L2, L3, L4, L5, L6, R = sy.symbols('L, h, H, L0, L1, L2, L3, L4, L5, L6, R')
L = 278e-3
h = 64e-3
H = 1104e-3
L0 = 270.35e-3
L1 = 69e-3
L2 = 364.35e-3
L3 = 69e-3
L4 = 374.29e-3
L5 = 10e-3
L6 = 368.3e-3
def HTM(alpha, a, d, theta):
    return sy.Matrix([
        [cos(theta), -sin(theta), 0, a],
        [sin(theta)*cos(alpha), cos(theta)*cos(alpha), -sin(alpha), -d*sin(alpha)],
        [sin(theta)*sin(alpha), cos(theta)*sin(alpha), cos(alpha), d*cos(alpha)],
        [0, 0, 0, 1],
    ])



DHparams = (
    (0, 0, 0, q[0, 0]),
    (-pi/2, L1, 0, q[1, 0]+pi/2),
    (pi/2, 0, L2, q[2, 0]),
    (-pi/2, L3, 0, q[3, 0]),
    (pi/2, 0, L4, q[4, 0]),
    (-pi/2, L5, 0, q[5, 0]),
    (pi/2, 0, 0, q[6, 0]),
)


T_RL_W0 = sy.Matrix([
    [-math.sqrt(2)/2, math.sqrt(2)/2, 0, -L,],
    [-math.sqrt(2)/2, -math.sqrt(2)/2, 0, -h,],
    [0, 0, 1, H,],
    [0, 0, 0, 1,],
])

T_0_RL = sy.Matrix([
    [1, 0, 0, 0,],
    [0, 1, 0, 0,],
    [0, 0, 1, L0,],
    [0, 0, 0, 1,],
])

Ts = [HTM(*dhparam) for dhparam in DHparams]

T_GR_7 = sy.Matrix([
    [1, 0, 0, 0,],
    [0, 1, 0, 0,],
    [0, 0, 1, L6,],
    [0, 0, 0, 1,],
])


### 変換前一覧 ###
T_all = [T_RL_W0, T_0_RL]
T_all += Ts
T_all.append(T_GR_7)


### 変換後 ###
for i, T in enumerate(T_all):
    if i == 0:
        T_abs = [T]
    else:
        T_abs.append(T_abs[i-1] @ T)


os = [T[0:3, 3:4] for T in T_abs]
Rxs = [T[0:3, 0:1] for T in T_abs]
Rys = [T[0:3, 1:2] for T in T_abs]
Rzs = [T[0:3, 2:3] for T in T_abs]

Jos = [o.jacobian(q) for o in os]
JRxs = [r.jacobian(q) for r in Rxs]
JRys = [r.jacobian(q) for r in Rys]
JRzs = [r.jacobian(q) for r in Rzs]



t = sy.Symbol("t")
q1 = sy.Function("q1")
q2 = sy.Function("q2")
q3 = sy.Function("q3")
q4 = sy.Function("q4")
q5 = sy.Function("q5")
q6 = sy.Function("q6")
q7 = sy.Function("q7")

dq = sy.Matrix(sy.MatrixSymbol('dq', 7, 1))

T_abs_ = []
for T in T_abs:
    T_ = T.subs([
        (q[0,0], q1(t)),
        (q[1,0], q2(t)),
        (q[2,0], q3(t)),
        (q[3,0], q4(t)),
        (q[4,0], q5(t)),
        (q[5,0], q6(t)),
        (q[6,0], q7(t)),
    ])
    T_abs_.append(T_)


os_ = [T[0:3, 3:4] for T in T_abs_]
Rxs_ = [T[0:3, 0:1] for T in T_abs_]
Rys_ = [T[0:3, 1:2] for T in T_abs_]
Rzs_ = [T[0:3, 2:3] for T in T_abs_]

q_ = sy.Matrix([
    [q1(t)],
    [q2(t)],
    [q3(t)],
    [q4(t)],
    [q5(t)],
    [q6(t)],
    [q7(t)],
])
Jos_ = [o.jacobian(q_) for o in os_]
JRxs_ = [r.jacobian(q_) for r in Rxs_]
JRys_ = [r.jacobian(q_) for r in Rys_]
JRzs_ = [r.jacobian(q_) for r in Rzs_]

Jos_dot_ = [sy.diff(J, t) for J in Jos_]
JRxs_dot_ = [sy.diff(J, t) for J in JRxs_]
JRys_dot_ = [sy.diff(J, t) for J in JRys_]
JRzs_dot_ = [sy.diff(J, t) for J in JRzs_]


Jos_dot = []
JRxs_dot = []
JRys_dot = []
JRzs_dot = []
for Js, newJs in zip((Jos_dot_, JRxs_dot_, JRys_dot_, JRzs_dot_), (Jos_dot, JRxs_dot, JRys_dot, JRzs_dot)):
    for J in Js:
        newJs.append(J.subs([
        (sy.Derivative(q1(t),t), dq[0, 0]),
        (sy.Derivative(q2(t),t), dq[1, 0]),
        (sy.Derivative(q3(t),t), dq[2, 0]),
        (sy.Derivative(q4(t),t), dq[3, 0]),
        (sy.Derivative(q5(t),t), dq[4, 0]),
        (sy.Derivative(q6(t),t), dq[5, 0]),
        (sy.Derivative(q7(t),t), dq[6, 0]),
        (q1(t), q[0, 0]),
        (q2(t), q[1, 0]),
        (q3(t), q[2, 0]),
        (q4(t), q[3, 0]),
        (q5(t), q[4, 0]),
        (q6(t), q[5, 0]),
        (q7(t), q[6, 0]),
    ]))


In [9]:
# 簡単化
os = [sy.simplify(e) for e in os]

In [6]:
from sympy.printing import cxxcode
from sympy.utilities.codegen import codegen

def gen_cpp_code(expr, name):
    code_txt = cxxcode(expr, assign_to="out", standard="c++17")
    with open(name+".cpp", "w") as f:
        f.write(code_txt)

def gen_c(expr, name, dir=""):
    [(c_name, c_code), (h_name, c_header)] = codegen(
        name_expr=(name, expr),
        language="C",
        project= name + "project",
        to_files=False
    )
    
    # f = open(dir+c_name, 'w')
    # f.write(c_code)
    # f.close()

    # f = open(dir+h_name, 'w')
    # f.write(c_header)
    # f.close()
    
    return c_code, c_header

names = ["W0", "BR"] + [str(i) for i in range(7)] + ["ee"]

# for i, o in enumerate(os):
#     gen_c(o, name="phi_"+names[i], dir="src/")
with open("src/Rxs.cpp", "w") as fc, open("src/Rxs.hpp", "w") as fh:
    for i, o in enumerate(Rxs):
        c, h = gen_c(o, name="Rx_"+names[i], dir="src/")
        fc.write(c)
        fh.write(h)
with open("src/Rys.cpp", "w") as fc, open("src/Rys.hpp", "w") as fh:
    for i, o in enumerate(Rxs):
        c, h = gen_c(o, name="Ry_"+names[i], dir="src/")
        fc.write(c)
        fh.write(h)
with open("src/Rzs.cpp", "w") as fc, open("src/Rzs.hpp", "w") as fh:
    for i, o in enumerate(Rxs):
        c, h = gen_c(o, name="Rz_"+names[i], dir="src/")
        fc.write(c)
        fh.write(h)
# for i, J in enumerate(Jos):
#     gen_c(J, name="Jo_"+names[i], dir="src/")
# for i, J in enumerate(JRxs):
#     gen_c(J, name="JRx_"+names[i], dir="src/")
# for i, J in enumerate(JRys):
#     gen_c(J, name="JRy_"+names[i], dir="src/")
# for i, J in enumerate(JRzs):
#     gen_c(J, name="JRz_"+names[i], dir="src/")
# for i, J in enumerate(Jos_dot):
#     gen_c(J, name="Jo_dot_"+names[i], dir="src/")
# for i, J in enumerate(JRxs_dot):
#     gen_c(J, name="JRx_dot_"+names[i], dir="src/")
# for i, J in enumerate(JRys_dot):
#     gen_c(J, name="JRy_dot_"+names[i], dir="src/")
# for i, J in enumerate(JRzs_dot):
#     gen_c(J, name="JRz_dot_"+names[i], dir="src/")





In [16]:
# pythonコード生成（クラス）
from sympy.printing.numpy import NumPyPrinter

names = ["W0", "BR"] + [str(i) for i in range(7)] + ["ee"]

common_w = "import numpy as np\nfrom math import cos as c\nfrom math import sin as s\nfrom math import tan as t\nfrom math import sqrt as sq\nfrom base import Base\n"



with open("src_py_/htm.py", "w") as f:
    f.write(common_w + "class HTM(Base):\n")
    for name, z in zip(names, os):
        numpy_word = "    def o_" + name + "(self, q):\n        return "
        f.write(numpy_word)
        f.write(NumPyPrinter().doprint(z))
        f.write("\n")

    for name, z in zip(names, Rxs):
        numpy_word = "    def rx_" + name + "(self, q):\n        return "
        f.write(numpy_word)
        f.write(NumPyPrinter().doprint(z))
        f.write("\n")

    for name, z in zip(names, Rys):
        numpy_word = "    def ry_" + name + "(self, q):\n        return "
        f.write(numpy_word)
        f.write(NumPyPrinter().doprint(z))
        f.write("\n")

    for name, z in zip(names, Rzs):
        numpy_word = "    def rz_" + name + "(self, q):\n        return "
        f.write(numpy_word)
        f.write(NumPyPrinter().doprint(z))
        f.write("\n")


with open("src_py_/Jos.py", "w") as f:
    f.write(common_w + "class Jo(Base):\n")
    for name, z in zip(names, Jos):
        numpy_word = "    def jo_" + name + "(self, q):\n        return "
        f.write(numpy_word)
        f.write(NumPyPrinter().doprint(z))
        f.write("\n")

with open("src_py_/JRxs.py", "w") as f:
    f.write(common_w + "class JRx(Base):\n")
    for name, z in zip(names, JRxs):
        numpy_word = "    def jrx_" + name + "(self, q):\n        return "
        f.write(numpy_word)
        f.write(NumPyPrinter().doprint(z))
        f.write("\n")

with open("src_py_/JRys.py", "w") as f:
    f.write(common_w + "class JRy(Base):\n")
    for name, z in zip(names, JRys):
        numpy_word = "    def jry_" + name + "(self, q):\n        return "
        f.write(numpy_word)
        f.write(NumPyPrinter().doprint(z))
        f.write("\n")

with open("src_py_/JRzs.py", "w") as f:
    f.write(common_w + "class JRz(Base):\n")
    for name, z in zip(names, JRzs):
        numpy_word = "    def jrz_" + name + "(self, q):\n        return "
        f.write(numpy_word)
        f.write(NumPyPrinter().doprint(z))
        f.write("\n")


with open("src_py_/Jo_dots.py", "w") as f:
    f.write(common_w + "class Jo_dot(Base):\n")
    for name, z in zip(names, Jos_dot):
        numpy_word = "    def jo_" + name + "_dot(self, q, dq):\n        return "
        f.write(numpy_word)
        f.write(NumPyPrinter().doprint(z))
        f.write("\n")

with open("src_py_/JRx_dots.py", "w") as f:
    f.write(common_w + "class JRx_dot(Base):\n")
    for name, z in zip(names, JRxs_dot):
        numpy_word = "    def jrx_" + name + "_dot(self, q, dq):\n        return "
        f.write(numpy_word)
        f.write(NumPyPrinter().doprint(z))
        f.write("\n")

with open("src_py_/JRy_dots.py", "w") as f:
    f.write(common_w + "class JRy_dot(Base):\n")
    for name, z in zip(names, JRys):
        numpy_word = "    def jry_" + name + "_dot(self, q, dq):\n        return "
        f.write(numpy_word)
        f.write(NumPyPrinter().doprint(z))
        f.write("\n")

with open("src_py_/JRz_dots.py", "w") as f:
    f.write(common_w + "class JRz_dot(Base):\n")
    for name, z in zip(names, JRzs):
        numpy_word = "    def jrz_" + name + "_dot(self, q, dq):\n        return "
        f.write(numpy_word)
        f.write(NumPyPrinter().doprint(z))
        f.write("\n")


def translate_hoge(original, done):
    with open(original, "r") as f, open(done, "w") as g:
        file_data = f.readlines()
        for line in file_data:
            line = line.replace('numpy', 'np').replace('1/2', '0.5').replace('(0.5)', '0.5')
            line = line.replace('np.cos', 'c').replace('np.sin', 's').replace('np.sqrt', 'sq')
            #line = line.replace('L', 'self.L').replace('h', 'self.h').replace('H', 'self.H')
            line = line.replace('import np as np', 'import numpy as np')
            line = line.replace('matself.h', 'math')
            g.write(line)


translate_hoge("src_py_/htm.py", "src_py/htm.py")
translate_hoge("src_py_/Jos.py", "src_py/Jos.py")
translate_hoge("src_py_/JRxs.py", "src_py/JRxs.py")
translate_hoge("src_py_/JRys.py", "src_py/JRys.py")
translate_hoge("src_py_/JRzs.py", "src_py/JRzs.py")
translate_hoge("src_py_/Jo_dots.py", "src_py/Jo_dots.py")
translate_hoge("src_py_/JRx_dots.py", "src_py/JRx_dots.py")
translate_hoge("src_py_/JRy_dots.py", "src_py/JRy_dots.py")
translate_hoge("src_py_/JRz_dots.py", "src_py/JRz_dots.py")

In [5]:
from sympy.printing.numpy import NumPyPrinter

names = ["W0", "BR"] + [str(i) for i in range(7)] + ["ee"]

common_w = "import numpy as np\nfrom math import cos as c\nfrom math import sin as s\nfrom math import tan as ta\nfrom math import sqrt as sq\n"

numba_word_q = "@njit(\"f8[:, :](f8[:, :])\")\n"
numba_word_q_dq = "@njit(\"f8[:, :](f8[:, :], f8[:, :])\")\n"

with open("src_py_/htm.py", "w") as f:
    f.write(common_w)
    for name, z in zip(names, os):
        numpy_word = "def o_" + name + "(q):\n    return "
        #f.write(numba_word_q)
        f.write(numpy_word)
        f.write(NumPyPrinter().doprint(z))
        f.write("\n")

    for name, z in zip(names, Rxs):
        numpy_word = "def rx_" + name + "(q):\n    return "
        #f.write(numba_word_q)
        f.write(numpy_word)
        f.write(NumPyPrinter().doprint(z))
        f.write("\n")

    for name, z in zip(names, Rys):
        numpy_word = "def ry_" + name + "(q):\n    return "
        #f.write(numba_word_q)
        f.write(numpy_word)
        f.write(NumPyPrinter().doprint(z))
        f.write("\n")

    for name, z in zip(names, Rzs):
        numpy_word = "def rz_" + name + "(q):\n    return "
        #f.write(numba_word_q)
        f.write(numpy_word)
        f.write(NumPyPrinter().doprint(z))
        f.write("\n")


with open("src_py_/Jos.py", "w") as f:
    f.write(common_w)
    for name, z in zip(names, Jos):
        numpy_word = "def jo_" + name + "(q):\n    return "
        #f.write(numba_word_q)
        f.write(numpy_word)
        f.write(NumPyPrinter().doprint(z))
        f.write("\n")

with open("src_py_/JRxs.py", "w") as f:
    f.write(common_w)
    for name, z in zip(names, JRxs):
        numpy_word = "def jrx_" + name + "(q):\n    return "
        #f.write(numba_word_q)
        f.write(numpy_word)
        f.write(NumPyPrinter().doprint(z))
        f.write("\n")

with open("src_py_/JRys.py", "w") as f:
    f.write(common_w)
    for name, z in zip(names, JRys):
        numpy_word = "def jry_" + name + "(q):\n    return "
        #f.write(numba_word_q)
        f.write(numpy_word)
        f.write(NumPyPrinter().doprint(z))
        f.write("\n")

with open("src_py_/JRzs.py", "w") as f:
    f.write(common_w)
    for name, z in zip(names, JRzs):
        numpy_word = "def jrz_" + name + "(q):\n    return "
        #f.write(numba_word_q)
        f.write(numpy_word)
        f.write(NumPyPrinter().doprint(z))
        f.write("\n")


with open("src_py_/Jo_dots.py", "w") as f:
    f.write(common_w)
    for name, z in zip(names, Jos_dot):
        numpy_word = "def jo_" + name + "_dot(q, dq):\n    return "
        #f.write(numba_word_q_dq)
        f.write(numpy_word)
        f.write(NumPyPrinter().doprint(sy.simplify(z)))
        f.write("\n")

with open("src_py_/JRx_dots.py", "w") as f:
    f.write(common_w)
    for name, z in zip(names, JRxs_dot):
        numpy_word = "def jrx_" + name + "_dot(q, dq):\n    return "
        #f.write(numba_word_q_dq)
        f.write(numpy_word)
        f.write(NumPyPrinter().doprint(sy.simplify(z)))
        f.write("\n")

with open("src_py_/JRy_dots.py", "w") as f:
    f.write(common_w)
    for name, z in zip(names, JRys):
        numpy_word = "def jry_" + name + "_dot(q, dq):\n    return "
        #f.write(numba_word_q_dq)
        f.write(numpy_word)
        f.write(NumPyPrinter().doprint(sy.simplify(z)))
        f.write("\n")

with open("src_py_/JRz_dots.py", "w") as f:
    f.write(common_w)
    for name, z in zip(names, JRzs):
        numpy_word = "def jrz_" + name + "_dot(q, dq):\n    return "
        #f.write(numba_word_q_dq)
        f.write(numpy_word)
        f.write(NumPyPrinter().doprint(sy.simplify(z)))
        f.write("\n")


def translate_hoge(original, done):
    with open(original, "r") as f, open(done, "w") as g:
        file_data = f.readlines()
        for line in file_data:
            line = line.replace('numpy', 'np').replace('1/2', '0.5').replace('(0.5)', '0.5')
            line = line.replace('np.cos', 'c').replace('np.sin', 's').replace('np.sqrt', 'sq')
            # line = line.replace(']])', ']], dtype=np.float64)')
            # line = line.replace('[0, 0, 0, 0, 0, 0, 0]', '[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]')
            # line = line.replace('[0]', '[0.0]').replace(' 0]],', ' 0.0]],').replace('[1]', '[1.0]').replace('[[0,', '[[0.0,').replace('0.0, 0],', '0.0, 0.0],')
            line = line.replace('import np as np', 'import numpy as np')
            g.write(line)


translate_hoge("src_py_/htm.py", "src_py_no_class/htm.py")
translate_hoge("src_py_/Jos.py", "src_py_no_class/Jos.py")
translate_hoge("src_py_/JRxs.py", "src_py_no_class/JRxs.py")
translate_hoge("src_py_/JRys.py", "src_py_no_class/JRys.py")
translate_hoge("src_py_/JRzs.py", "src_py_no_class/JRzs.py")
translate_hoge("src_py_/Jo_dots.py", "src_py_no_class/Jo_dots.py")
translate_hoge("src_py_/JRx_dots.py", "src_py_no_class/JRx_dots.py")
translate_hoge("src_py_/JRy_dots.py", "src_py_no_class/JRy_dots.py")
translate_hoge("src_py_/JRz_dots.py", "src_py_no_class/JRz_dots.py")